# Bing Chat version below

Doesn't really work.

In [2]:
meanITI = 12
numrewards = 500

# anccr model parameters
samplingperiod = 0.2
alpha_anccr = {}
alpha_anccr["exponent"] = 0.1
alpha_anccr["init"] = 0.25
alpha_anccr["min"] = 0.02
alpha_r = 0.2
w = 0.5          
k = 1
minimumrate = 10 ** -3
maximumjitter = 0.1
beta = [1]  # Check what beta does in calculate_anccr
threshold = 0.6
T = meanITI*10  # have very large T

nIter = 100
rwrsp = np.zeros((numrewards,nIter))

In [10]:
import numpy as np

def simulateBackgroundRewards(numrewards, rewITI, rewlabel, rewmag, truncation):
    """
    SIMULATEBACKGROUNDREWARDS outputs an eventlog with exclusively background
    rewards. Inputs may be scalar or arrays depending on the number of unique
    background rewards specified.
    """
    # Set upper limit on ITI (only used if truncation == 1)
    maxrewITI = 3 * rewITI
    # Initialize empty eventlog
    eventlog = np.empty((numrewards, 3))
    eventlog[:] = np.nan
    # Keep track of reward indices
    running_idx = 0
    # If multiple unique rewards, loop instance for each type
    for irw in range(1, rewlabel+1):
        # Start simulation time at 0
        running_time = 0
        # For each reward (irw) simulate 'numrewards' timesteps
        for i in range(1, numrewards+1):
            running_idx += 1
            # Generate exp. dist. timestep for current reward instance
            new_ts = np.random.exponential(rewITI, 4)
            if truncation == 1:  # If capping ITI at maxITI
                while new_ts > maxrewITI:
                    # Simulate a new ts if maxITI is exceeded
                    new_ts = np.random.exponential(rewITI, 4)
            # Update eventlog
            eventlog[running_idx - 1, 0] = rewlabel  # Column 1 = label
            eventlog[running_idx - 1, 1] = new_ts + running_time  # " 2 = ts
            eventlog[running_idx - 1, 2] = rewmag  # " 3 = rew. magnitude
            running_time += new_ts  # Update running time

    # Resort eventlog by timestamp
    eventlog = eventlog[eventlog[:, 1].argsort()]

    # Once any reward reaches to numrewards, finish the session
    lasttrial = min([np.where(eventlog[:,0]==x)[0][-1] for x in rewlabel])
    eventlog = eventlog[:lasttrial+1,:]
    
    return eventlog

In [11]:
simulateBackgroundRewards(numrewards, meanITI, 1, 1, 0)

ValueError: setting an array element with a sequence.